<a href="https://colab.research.google.com/github/udaykondreddy/Code-for-learn-machinelearning/blob/master/text_classification/text_classification_BERT_scikit-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Learn more from [here](https://instagram.com/learn.machinelearning)

In [ ]:
# Getting data from kaggle 

#import library
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=8af8d5364ef999c6eb14245922506f468790dfa2284b2c92aad5a519546e787c
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [ ]:
!mkdir ~/.kaggle

In [ ]:
#upload your kaggle API token
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"udaykiran3207","key":"5fae2743dab6e2250e508eeee7681af3"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle 
!!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

['Downloading imdb-dataset-of-50k-movie-reviews.zip to /content',
 '',
 '  0% 0.00/25.7M [00:00<?, ?B/s]',
 ' 19% 5.00M/25.7M [00:00<00:01, 14.5MB/s]',
 '',
 '100% 25.7M/25.7M [00:00<00:00, 64.0MB/s]']

In [ ]:
#Unzip data
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
#Install transformers library (by huggingface)
!pip install transformers

     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 3.0MB 19.5MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3fcd9b07ba407f24cdd2733e4131ef0949d4d478aaa76b021aa616e5d1f7a34c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
import warnings
warnings.filterwarnings('ignore')
import re

In [ ]:
#load data
data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#Check how many samples are there for each class
data["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
#Pre process data
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

reviews = []
sentences = list(data['review'])
for sen in sentences:
    text = preprocess_text(sen)
    #reduce the text to 512 words
    text = text.split(" ")
    if len(text)>512:
      text = text[:511]
    text = " ".join(text)
    reviews.append(text)

In [ ]:
len(reviews[1000].split())

119

In [ ]:
data["review"] = reviews

In [ ]:
#Map positive to 1 and negative to 0
y = data["sentiment"]
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [ ]:
#Load pre-trained model (BERT)
model, tokenizer, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

In [ ]:
# Load pretrained model/tokenizer
tokenizer = tokenizer.from_pretrained(pretrained_weights)
model = model.from_pretrained(pretrained_weights)

In [ ]:
# Preprocessing the data into the format of BERT
'''
It is a 3 step process 
1. Break words into tokens
2. Add[CLS] and [SEP] tokens
3. Substitute words with tokens

#You can do all these in one line of code
'''
tokenized_data = data["review"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation=True)))

In [ ]:
#Padding data to make every sentence have the same length
max_len = 0
for i in tokenized_data.values:
    if len(i) > max_len:
        max_len = len(i)

padded_data = np.array([i + [0]*(max_len-len(i)) for i in tokenized_data.values])
padded_data.shape

In [ ]:
#If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is
attention_mask = np.where(padded_data != 0, 1, 0)
attention_mask.shape

In [ ]:
#Now let's get the word embedings from the BERT model
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    embeddings = model(input_ids, attention_mask=attention_mask)

In [ ]:
embeddings.shape()

In [ ]:
#Lets split the data into train and test
train_features, test_features, train_labels, test_labels = train_test_split(features, y)

In [ ]:
#huper parameter tuning to find the best model
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)


In [ ]:
#evaluate the model
lr_clf.score(test_features, test_labels)